## Previously on....
### Get some batch

In [2]:
from process import get_data
from torchtext import data

In [ ]:
data_path = "dataset/"
file_name = "train."

In [3]:
txt_en, train_en = get_data(file_path = data_path + file_name + 'en',
                           field_name = 'en')

train_loader = data.Iterator(train_en, batch_size = 3,
                            device = None, # if using GPU, type "cuda" 
                            repeat = False)

for batch in train_loader:
    break

a = batch.en
b = a[1]

print(b)

In [8]:
del txt_en, train_en, train_loader, batch.en

In [25]:
b

tensor([    2, 23520,    20,    33,   168,  2678,     3,     1,     1,     1])

### Emedding and Positional Encoding

In [ ]:
from embedding import Embeddings, PositionalEncoding

In [ ]:
emb = Embeddings(d_model = 128, vocab = 25000)

x = emb(b).unsqueeze(0)

In [12]:
PE = PositionalEncoding(d_model = 128, dropout = .1)

x = PE(x)

In [24]:
x.shape

torch.Size([1, 10, 128])

# Model Architecture

In [84]:
import torch
import torch.nn as nn
import math
import numpy as np
import copy
import torch.nn.functional as F

## Attention

masked_fill(mask, value) : mask 자리에 value값 집어넣기

input의 뒤에서 부터 index를 따옴으로써, Multi-Head로 나눌때도 활용 가능

In [62]:
query, key, value = x, x, x

In [35]:
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [91]:
def attention(query, key, value, mask = None, dropout = None):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    
    p_attn = F.softmax(scores, dim = -1)
    
    if dropout is not None:
        p_attn = dropout(p_attn)
    
    return torch.matmul(p_attn, value), p_attn

In [93]:
output, attn = attention(query, key, value)

In [94]:
print(output.shape, attn.shape)

torch.Size([1, 10, 128]) torch.Size([1, 10, 10])


In [103]:
attn[0,9]

tensor([5.2112e-07, 1.4196e-02, 7.8867e-07, 1.1218e-03, 1.7305e-15, 9.8425e-01,
        9.6199e-05, 5.3271e-05, 7.3586e-05, 2.1246e-04],
       grad_fn=<SelectBackward>)

In [71]:
nbatches = query.size(0)
d_k = query.size(-1)

In [70]:
key.transpose(-2, -1).shape

torch.Size([1, 128, 10])

In [75]:
scores = torch.matmul(query, key.transpose(-2, -1)) \
            / math.sqrt(d_k)

In [82]:
mask = torch.ones(scores.size())

In [80]:
mask[]

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [83]:
scores.masked_fill(mask == 0, -1e9)

tensor([[[ 1.3202e+03,  1.5606e+02,  7.3210e+01, -1.7394e+01,  3.3429e+01,
           5.2500e+00,  1.2682e+02,  1.4988e+00, -2.0276e-01, -9.3661e-02],
         [ 1.5606e+02,  1.5846e+03,  5.4260e+01, -1.8045e+02, -1.2955e+02,
           7.2400e+00,  2.7251e+02,  1.7234e+01,  1.1637e+01,  1.0119e+01],
         [ 7.3210e+01,  5.4260e+01,  1.7286e+03, -1.0054e+02,  3.5896e+02,
           4.6190e+01,  8.0023e+01,  1.2181e+00,  4.7280e+00,  3.2072e-01],
         [-1.7394e+01, -1.8045e+02, -1.0054e+02,  1.5374e+03,  1.1725e+02,
           2.0427e+01, -2.0561e+02,  6.8445e+00,  7.9031e+00,  7.5808e+00],
         [ 3.3429e+01, -1.2955e+02,  3.5896e+02,  1.1725e+02,  1.8019e+03,
          -6.8380e+00,  1.5560e+02, -1.2167e+01, -1.3067e+01, -1.9617e+01],
         [ 5.2500e+00,  7.2400e+00,  4.6190e+01,  2.0427e+01, -6.8380e+00,
           1.8991e+03,  1.0492e+02,  1.7525e+01,  1.0650e+01,  1.4358e+01],
         [ 1.2682e+02,  2.7251e+02,  8.0023e+01, -2.0561e+02,  1.5560e+02,
           1.0492e+

In [85]:
p_attn = F.softmax(scores, dim = -1)

In [90]:
p_attn.shape

torch.Size([1, 10, 10])

In [89]:
torch.matmul(p_attn, value).shape

torch.Size([1, 10, 128])

## Multi-head Attention

tensor.view(*shape) : Returns a new tensor with the same data but of a different shape.
여기선 Tensor Slice를 위해 사용됨

Assert *arg, *error : *arg == False 일때 반환할 *error 선언

tensor.contiguous() : makes a copy of tensor so the order of elements would be same as if tensor of same shape created from scratch [doc](https://stackoverflow.com/questions/48915810/pytorch-contiguous)

? MHA Runtime Error 왜 Trace가 남지

In [122]:
h = 8
d_model = 128
d_k = d_model // h

In [319]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout = .1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p = dropout)
    
    def forward(self, query, key, value, mask = None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Linear Projection 태우고 나서 Slicing by h
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1,2)
                             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) 각각에 Attention 적용
        x, self.attn = attention(query, key, value, mask = mask,
                                 dropout = self.dropout)
        
        # 3) 각 Output 다시 모은 후 Final Linear Projection
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        
        return self.linears[-1](x)

In [333]:
MHA = MultiHeadedAttention(8, 128)

In [331]:
MHA(x, x, x)

tensor([[[-0.7084, -2.5016, -1.9505,  ...,  3.3102, -0.9901,  1.1454],
         [ 1.3478, -2.0266, -3.4824,  ...,  1.1152,  1.3645, -1.0921],
         [-2.2603,  1.0380, -0.4749,  ..., -3.6007,  4.4375, -0.0069],
         ...,
         [ 0.1151, -3.7868,  2.4041,  ..., -0.3625,  1.9119,  3.1872],
         [ 0.4620, -0.0891,  1.6847,  ...,  1.8731, -1.1306,  3.8335],
         [ 0.2936, -0.7766, -0.0127,  ..., -2.0389, -2.0005, -5.2767]]],
       grad_fn=<AddBackward0>)

In [328]:
clones(MHA, 6)

ModuleList(
  (0): MultiHeadedAttention(
    (linears): ModuleList(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): Linear(in_features=128, out_features=128, bias=True)
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): Linear(in_features=128, out_features=128, bias=True)
    )
    (dropout): Dropout(p=0.1)
  )
  (1): MultiHeadedAttention(
    (linears): ModuleList(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): Linear(in_features=128, out_features=128, bias=True)
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): Linear(in_features=128, out_features=128, bias=True)
    )
    (dropout): Dropout(p=0.1)
  )
  (2): MultiHeadedAttention(
    (linears): ModuleList(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): Linear(in_features=128, out_features=128, bias=True)
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): Linear(in_features=128, o

In [332]:
# after one MHA(x, x, x)
clones(MHA, 6)

RuntimeError: Only Tensors created explicitly by the user (graph leaves) support the deepcopy protocol at the moment

In [297]:
linear_list = clones(nn.Linear(d_model, d_model), 4)

In [298]:
linear_list[0].weight

Parameter containing:
tensor([[ 0.0031, -0.0645, -0.0335,  ..., -0.0263,  0.0064, -0.0342],
        [ 0.0676,  0.0665, -0.0368,  ..., -0.0427,  0.0705,  0.0263],
        [ 0.0185, -0.0052, -0.0064,  ...,  0.0766, -0.0430,  0.0225],
        ...,
        [ 0.0341,  0.0454,  0.0349,  ..., -0.0475,  0.0723,  0.0621],
        [-0.0743, -0.0566,  0.0392,  ..., -0.0472,  0.0438,  0.0549],
        [-0.0020,  0.0203,  0.0125,  ..., -0.0259, -0.0612, -0.0148]],
       requires_grad=True)

In [299]:
print(mask.shape, mask.unsqueeze(1).shape)

torch.Size([1, 10, 10]) torch.Size([1, 1, 10, 10])


1. Linear projection of x => Attention 모듈 인풋으로 가공

In [300]:
linear_list[0](query).view(nbatches, -1, h, d_k).shape

torch.Size([1, 10, 8, 16])

In [127]:
linear_list[0](query).view(nbatches, -1, h, d_k).transpose(1,2).shape

torch.Size([1, 8, 10, 16])

In [128]:
assert 4 % 3 == 0, "d_model should be divided by #(h)"

AssertionError: d_model should be divided by #(h)

In [129]:
qu1, ke1, va1 = [l(x).view(nbatches, -1, h, d_k).transpose(1,2)
                for l, x in zip(linear_list, (query, key, value))]

In [131]:
qu1.size()

torch.Size([1, 8, 10, 16])

2. apply attention

In [149]:
x, m_attn = attention(qu1, ke1, va1)

In [150]:
print(x.shape,m_attn.shape)

torch.Size([1, 8, 10, 16]) torch.Size([1, 8, 10, 10])


In [151]:
# transpose function 이해
torch.sum(x.transpose(2,1) == x.transpose(1,2))

tensor(1280)

3. concat and final linear layer

In [155]:
x = x.transpose(1,2).contiguous().view(nbatches, -1, h * d_k)

In [156]:
x.shape

torch.Size([1, 10, 128])

In [157]:
linear_list[-1](x)

tensor([[[  2.4052,  -0.5851,  -2.8357,  ...,   5.4792,  -6.7533,   7.0882],
         [ -1.5129,   2.2423,  -6.8953,  ...,  -0.3565,  -2.4764,   0.9368],
         [ -1.8745,  -3.9184,   7.1304,  ...,   3.8954, -10.8487,   1.0484],
         ...,
         [ -8.4622,  -2.9153,   9.2156,  ...,  -5.1566,  -0.7907,  -2.9662],
         [  1.0004,  -2.1786,  -0.5619,  ...,  -1.1994,  -1.9616,  -1.2122],
         [ -7.6547,  -7.7237,   3.3475,  ...,  -4.2863,  -3.0021,  -8.7714]]],
       grad_fn=<AddBackward0>)

## Position-wise Feed-Forward Networks

In [301]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout = .1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [302]:
FFN = PositionwiseFeedForward(128, 1024)

## Layer Normalization

In [159]:
x.mean(-1, keepdim = True).shape

torch.Size([1, 10, 1])

In [303]:
class LayerNorm(nn.Module):
    def __init__(self, size, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.eps = eps
        self.a_2 = nn.Parameter(torch.ones(size))
        self.b_2 = nn.Parameter(torch.zeros(size))
    
    def forward(self, x):
        mean = x.mean(-1, keepdim = True)
        std = x.std(-1, keepdim = True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [304]:
LN = LayerNorm(128)

In [305]:
LN(x)

tensor([[[ 2.0053, -0.0690, -0.5193,  ...,  1.4481, -1.7137, -0.1258],
         [ 1.8503, -0.1294, -0.4929,  ..., -0.5587,  0.1448,  1.6142],
         [-0.6065, -0.5054, -1.0229,  ...,  0.8431,  0.8444, -0.4091],
         ...,
         [ 0.4244,  0.2607, -0.3858,  ..., -1.2285,  0.0095, -0.9126],
         [ 0.5966,  1.3276, -0.4592,  ..., -1.6051,  1.1125, -1.1722],
         [-0.5457, -0.4284,  0.0458,  ..., -0.3410, -0.1303,  0.3102]]],
       grad_fn=<AddBackward0>)

## Mask

In [ ]:
def subsequent_mask(size):
    attn_shape = (1, size, size)
    mask = torch.triu(torch.ones(attn_shape, dtype = torch.float), diagonal = 1)
    return mask == 0

In [351]:
attn_shape = (1, 128, 128)

In [367]:
torch.triu(torch.ones(attn_shape, dtype = torch.float), diagonal = 1) == 0

tensor([[[1, 0, 0,  ..., 0, 0, 0],
         [1, 1, 0,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 1, 0, 0],
         [1, 1, 1,  ..., 1, 1, 0],
         [1, 1, 1,  ..., 1, 1, 1]]], dtype=torch.uint8)

## Encoder

In [306]:
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout = .1):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, sublayer):
        return self.norm(x + self.dropout(sublayer(x)))

In [307]:
sublayer = SublayerConnection(128)

In [308]:
EL_ = sublayer(x, lambda x: MHA(x, x, x)).shape

In [344]:
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout = .1):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size
        
    def forward(self, x, mask = None):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [345]:
EL = EncoderLayer(128, MHA, FFN)

In [346]:
class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layer = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return x

In [347]:
ENCODER = Encoder(EL, 6)

## Decoder

In [348]:
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout = .1):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x :self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)
        

In [349]:
class Decoder(nn.Module):
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layer = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return x

## Transformer Model
nn.Sequential(*modules) : A sequential container. Modules will be added to it 

In [387]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generaotr = generator
    
    def forward(self, src, tgt, src_mask, tgt_mask):
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src, src_mask))
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.deocder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [393]:
class Generator(nn.Module):
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)
        
    def forward(self, x):
        return F.log_softmax(self.proj(x), dim = -1)

In [396]:
def Transformer(src_vocab, tgt_vocab, N=6, d_model = 512, d_ff = 2048,
               h=8, dropout = .1):
    
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

In [397]:
tmp_model = Transformer(10, 10, 2)

In [398]:
tmp_model

EncoderDecoder(
  (encoder): Encoder(
    (layer): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0): Linear(in_features=512, out_features=512, bias=True)
            (1): Linear(in_features=512, out_features=512, bias=True)
            (2): Linear(in_features=512, out_features=512, bias=True)
            (3): Linear(in_features=512, out_features=512, bias=True)
          )
          (dropout): Dropout(p=0.1)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=512, out_features=2048, bias=True)
          (w_2): Linear(in_features=2048, out_features=512, bias=True)
          (dropout): Dropout(p=0.1)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
          )
          (1): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
       

In [371]:
x.shape

torch.Size([1, 10, 128])

In [373]:
linear_ = nn.Linear(128, 25000)

In [377]:
y = F.log_softmax(linear_(x), dim = -1)

In [378]:
y.shape

torch.Size([1, 10, 25000])

In [382]:
torch.argmax(y, dim = -1)

tensor([[13523, 21630, 11863,   253,  5077, 16131, 11914,  7700, 12435, 14339]])